---
# Downscaling - SL2P 

This notebook contains code blocks to run SL2P based on different inputs, which are as follows:
1. NEON upscaled to 10m with the SL2P20 for prediction (Reference)
2. Treatment N - S2NEON upscaled to 20m with the SL2P20 for prediction(null hypothesis )
3. Treatment A - S2NEON B,G,R,NIR with the SL2P10 for prediction
4. Treatment B - S2NEON downscaled to 10m using APTRK with SL2P20 for prediction
5. Treatment C - S2NEON with ALR for prediction
6. Statistical analysis
---

In [2]:
import ee
import time
import math
import csv
import json
import os
import numpy as np
import pandas as pd
import folium ; from folium import plugins
import matplotlib.pyplot as plt
import scipy ; from scipy import stats
import scipy.io as sio
import sklearn as skl ; from sklearn import linear_model ; from sklearn import preprocessing
import tensorflow as tf
import pickle
from collections import OrderedDict
from sklearn.metrics import mean_squared_error 
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import r2_score
# import custom modules (files must be in same directory as this notebook)
import feature_collections as fc
import image_bands as ib
import wrapper_nets as wn
import ee_functions as ee_func
import ALR_functions as alr

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AbUR2VORKatKnaqWhGzYGXDXwTs3Aj4P6qS2HUG95rP4PL5vqAfyqQGXPI0



Successfully saved authorization token.


---
# Prelim: Define dictionaries

In [26]:
# -----------------------------------------------------
# set parameters based on user-defined parameters above
# -----------------------------------------------------
outputParams = {
    'fAPAR': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'fCOVER': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'LAI': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 8
    }
}
outputName = 'LAI'
outputScale = outputParams[outputName]['outputScale']
outputOffset = outputParams[outputName]['outputOffset']
outputMax = outputParams[outputName]['outputMax']
responseBand = 'estimate'+outputName

In [27]:
COLLECTION_OPTIONS = {
    # Sentinel 2 using 20 m bands:
    'COPERNICUS/S2_SR': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'COPERNICUS/S2_SR_10m': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'NEON_Sim_10m': {
      "name": 'NEON_Sim_10m',
      "description": 'NEON Simulated',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
     'NEON_Sim': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
}

VIS_OPTIONS = {
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],   
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
          "NEON_Sim_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "NEON_Sim": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    }
}

In [28]:
### for NEON bands
def  renameBands_NEON(image):
    bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14','b15','b16']
    new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
    return image.select(bands).rename(new_bands)

In [29]:
## ALR function for estimation,after feature selection, a simple random forest estiamte applied
def func_ALR(temp_image, responsedBand, outputName, mapBounds):
   
    # inputImage = ee.Image(temp_image).select(1,2,3,7,22,23,27,28,29,30,31,32)
    # inputImage = ee.Image(temp_image).select(1,2,3,7,16,17,18,19,20,21)
    # inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'QA60', 'date', 'estimate'+outputName, 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'estimate'+outputName, 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    # inputImage = inputImage.rename(inputImage_bands)
    inputImage = ee.Image(temp_image).select(inputImage_bands)
    input_VI_definition = ee.List([
                                   "GI      = b('B3')/b('B4')",                               
                                   "SGI     = b('B8')/b('B4')",                                
                                   "GVI     = (b('B8')/b('B3'))-1",                             
                                   "NDVI3   = ((b('B8')-b('B4'))/(b('B8')))+b('B4')",                                
                                   "NDVI    = (b('B8')-b('B4'))/(b('B8')+b('B4'))",
                                   "GNDVI   = (b('B8')-b('B3'))/(b('B8')+b('B3'))",                                
                                   "NDGI    = (b('B3')-b('B4'))/(b('B3')+b('B4'))",                                 
                                   "EVI     = 2.5*((b('B8')-b('B4'))/(b('B8')+6*b('B4')-7.5*b('B3')+1))",
                                   "EVI2    = 2.5*((b('B8')-b('B4'))/(b('B8')+2.4*b('B4')+1))",
                                   "RDVI    = (b('B8')-b('B4'))/((b('B8')+b('B4'))**0.5)",
                                   "MSR     = ((b('B8')/b('B4'))-1)/((b('B8')/b('B4'))**0.5+1)",                            
                                   "MSAVI2  = 0.5*(2*b('B8')+1-((2*b('B8')+1)**2-8*(b('B8')-b('B4')))**0.5)",                                
                                   "NLI     = ((b('B8')**2)-b('B4'))/((b('B8')**2)+b('B4'))"])

    # names of bands to pass to ALR method (excluding metadata and other non-spectral bands)
    input_bandNames = ['B2', 'B3', 'B4', 'B8', 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI']
    
    def format_image(image, image_bands, response_band, VI_definition):
        image = ee.Image(image)
        image_bands = ee.List(image_bands)
        response_band = ee.String(response_band)
        VI_definition = ee.List(VI_definition)
        
        # image_bands specifices a list of the names of the bands used in defining the expressions for VIs in VI_definition
        image = image.rename(image_bands).toDouble()
        
        # Generate an ImageCollection from a list of expressions defining a set of VIs using the bands available in the image
        VIimageCollection = ee.ImageCollection(VI_definition.map(lambda expr: image.expression(expr)))
        VIimage = VIimageCollection.toBands().regexpRename("[0-9]+_", "")
        
        # Reorder the bands in the image so the response band is the first band in the image
        feature_bands = image_bands.remove(response_band)
        
        return ee.Image(image.select(response_band).addBands(VIimage).addBands(image.select(feature_bands)))

    
    inputImage = format_image(inputImage, inputImage_bands, responseBand, input_VI_definition)
    
    def scale_image(image, response_band):
        image = ee.Image(image)
        response_band = ee.String(response_band)
        
        def get_num_pixels(image):
    
            # get image height
            def get_height(image):
                height = image.getInfo()["bands"][0]["dimensions"][0]
                return height

            # get image width
            def get_width(image):
                width = image.getInfo()["bands"][0]["dimensions"][1]
                return width

            image_height = get_height(image)
            image_width = get_width(image)
            image_pixels = image_height*image_width

            return image_pixels
        
        image_pixels = ee.Number(get_num_pixels(image))
        
        # Set up lists containing the input/feature bands in the image
        bandList = image.bandNames()
        featureList = bandList.remove(response_band)
        num_bands = bandList.length()
        num_features = featureList.length()
        
        # We will be using the reduceRegion() function on images from Earth Engine, 
        # which will process up to a specified number of pixels from the image to generate the outputs of the reducer
        max_pixels = image_pixels.min(10000000)
        # best_effort = ee.Algorithms.If(image_pixels.gt(max_pixels), True, False)
        
        # Set default projection and scale using the response band
        defaultScale = image.select(response_band).projection().nominalScale()
        defaultCrs = image.select(response_band).projection().crs()
        image = image.setDefaultProjection(crs=defaultCrs, scale=defaultScale)
        
        # Center all of the bands in the image for LARs
        # We will centre the sampled data later as well as reduceRegion() is not precise enough
        meanImage = image.subtract(image.reduceRegion(reducer=ee.Reducer.mean(), \
                                    scale=defaultScale, bestEffort=True, maxPixels=max_pixels).toImage(bandList))
        
        # Separate the image into features (X) and response (y) as we need to standardize the input features
        X = meanImage.select(featureList)
        y = meanImage.select(response_band)
        
        # Standardize the input features
        X = X.divide(X.reduceRegion(reducer=ee.Reducer.stdDev(), bestEffort=True, maxPixels=max_pixels).toImage(featureList))
        
        return X.addBands(y)

    
    scaledImage = scale_image(inputImage, responseBand)
    
    
    def ee_LARS(input_image, input_bandNames, response_bandName, num_nonzero_coefficients, num_samples):
        image = ee.Image(input_image)
        feature_list = ee.List(input_bandNames)
        response_band = ee.String(response_bandName)
        full_band_list = ee.List(feature_list).add(response_band)
        num_nonzero_coefficients = ee.Number(num_nonzero_coefficients)
        num_samples = ee.Number(num_samples)
        def get_num_pixels(image):
    
            # get image height
            def get_height(image):
                height = image.getInfo()["bands"][0]["dimensions"][0]
                return height

            # get image width
            def get_width(image):
                width = image.getInfo()["bands"][0]["dimensions"][1]
                return width

            image_height = get_height(image)
            image_width = get_width(image)
            image_pixels = image_height*image_width

            return image_pixels
        image_pixels = ee.Number(get_num_pixels(image))
        
        # Randomly sample pixels in the image at native resolution into a FeatureCollection
        input_collection = image.sample(numPixels=num_samples.min(image_pixels))
        n = input_collection.size()
        m = feature_list.length()
        
        # Use an aggregate array function over the FeatureCollection and map the function over each feature in the band list
        # to generate a dictionary of all of the samples retrieved
        inputs = ee.Dictionary.fromLists(full_band_list, full_band_list.map(lambda feature: input_collection.aggregate_array(feature)))
        
        # Although we may call our scale_image function on the input image, the reduceRegion() function used to determine the mean
        # and standard deviation of each band in the image over the entire region is not precise enough over a large image
        # so we must recenter all of the bands in the image and now we can also normalize (L2 norm) each input feature as required
        # by the LARs algorithm
        
        # Use an aggregate_mean function over the feature collection to get the mean of each band
        input_means = ee.Dictionary.fromLists(full_band_list, full_band_list.map(lambda feature: input_collection.aggregate_mean(feature)))

        def centre_inputs(key, value):
            key_mean = input_means.getNumber(key)
            return ee.List(value).map(lambda sample: ee.Number(sample).subtract(key_mean))
        
        
        # Center bands by mapping over the list of features and then a subtracting over the list of samples for each band
        inputs = inputs.map(centre_inputs)

        # Separate the response variable samples into its own vector
        y = inputs.toArray([response_band]).reshape([-1,1])

        # Remove response band from the feature collection by selecting only bands in the feature list
        inputs = inputs.select(feature_list)
        
        # Generate a dictionary of all of the L2 norms of the input features using a custom mapped function
        input_norms = inputs.map(lambda key, value: ee.Number(ee.List(value).map(lambda sample: ee.Number(sample).pow(2)).reduce(ee.Reducer.sum())).pow(0.5))

        def norm_inputs(key, value):
            key_norm = input_norms.getNumber(key)
            return ee.List(value).map(lambda sample: ee.Number(sample).divide(key_norm))
        
        # Normalize all of the features by mapping a function over the list of features
        # and then map a division over the list of all of the samples of the feature
        inputs = inputs.map(norm_inputs)
        
        # Generate the array of samples using the dictionary
        X = inputs.toArray(feature_list).transpose()

        # Find the first best predictor of the response to initialize the main LARs loop
        initial_prediction = ee.Array(ee.List.repeat([0], n))
        c = X.transpose().matrixMultiply(y.subtract(initial_prediction))
        c_abs = c.abs()
        C_maxLoc = c_abs.project([0]).argmax()
        add_feature = C_maxLoc.getNumber(0)
        A = ee.List([add_feature])
        
        # Create a dicitionary of initial inputs to pass into the main LARs iterative loop
        # The iterate function in Earth Engine processes each iteration as a tree of iterations with no access to any variables
        # from previous iterations (only those that are passed to the next iteration)
        # so we must pass both the current prediction and the active set of features (with non-zero coefficients), A
        initial_inputs = ee.Dictionary({'prediction': initial_prediction, 'A': A})

        def LARs_regression(iteration, inputs):
            inputs = ee.Dictionary(inputs)

            # Find the active set of features, A (predictors with non-zero coefficients)
            A = ee.List(inputs.get('A'))
            # A_list is an array used to mask the full array of input samples and the correlation vector
            A_list = ee.Array(ee.List.sequence(0, m.subtract(1))\
                              .map(lambda index: A.contains(index)).replaceAll(False, 0).replaceAll(True, 1)).reshape([-1,1])

            # The following matrix algebra determines the next most correlated variable, or the next best predictor considering the
            # current features in the active set, A, as well as the magnitude to adjust the prediction vector to ensure all of the
            # features in the active set are equally correlated to response vector
            prediction = inputs.getArray('prediction')
            c = X.transpose().matrixMultiply(y.subtract(prediction))
            c_abs = c.abs()
            C_max = c_abs.get(c_abs.argmax())
            s_A = c.divide(c_abs).mask(A_list)
            X_A = X.mask(A_list.transpose())
            G_Ai = X_A.transpose().matrixMultiply(X_A).matrixInverse()
            G1 = G_Ai.matrixMultiply(s_A)
            A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
            w_A = G1.multiply(A_A)
            u_A = X_A.matrixMultiply(w_A)
            a = X.transpose().matrixMultiply(u_A)
            a = a.project([0])
            c = c.project([0])

            def compute_gammaArray(index_j):
                minus_j = C_max.subtract(c.get([index_j])).divide(A_A.subtract(a.get([index_j])))
                plus_j = C_max.add(c.get([index_j])).divide(A_A.add(a.get([index_j])))
                return ee.List([minus_j, plus_j]).filter(ee.Filter.gte('item', 0)).reduce(ee.Reducer.min())

            A_c = ee.List.sequence(0, m.subtract(1)).removeAll(A)
            gammaArray = A_c.map(compute_gammaArray)
            gamma = gammaArray.reduce(ee.Reducer.min())
            min_location = gammaArray.indexOf(gamma)
            add_feature = A_c.getNumber(min_location)

            # Update active set of variables with next best predictor from non-active set and update prediction vector
            A = A.add(add_feature)
            prediction = prediction.add(u_A.multiply(gamma))

            return ee.Dictionary({'prediction': prediction, 'A': A})


        # The final iteration of LARs (if selecting all input variables) requires a different method to determine magnitude for
        # adjusting the magnitude of the prediction vector, as the regular LARs iteration relies on variables in non-active set
        # In the final iteration there will be no variables in the non-active set, so the method will not work
        def LARs_final_iteration(iteration, inputs):
            inputs = ee.Dictionary(inputs)
            A = ee.List(inputs.get('A'))

            prediction = inputs.getArray('prediction')
            c = X.transpose().matrixMultiply(y.subtract(prediction))
            c_abs = c.abs()
            C_max = c_abs.get(c_abs.argmax())        

            s_A = c.divide(c_abs)
            G_Ai = X.transpose().matrixMultiply(X).matrixInverse()
            G1 = G_Ai.matrixMultiply(s_A)
            A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
            w_A = G1.multiply(A_A)
            u_A = X.matrixMultiply(w_A)

            gamma = C_max.divide(A_A)
            prediction = prediction.add(u_A.multiply(gamma))

            return ee.Dictionary({'prediction': prediction, 'A': A})

        # Actually carrying out the iterations by iterating over a placeholder list (sequence from 1 to the number of non-zero
        # variables that the user wishes to select as predictors for the response)
        iterations = ee.List.sequence(1, m.subtract(1).min(num_nonzero_coefficients))
        penultimate_outputs = iterations.iterate(LARs_regression, initial_inputs)
        final_outputs = ee.Dictionary(ee.Algorithms.If(num_nonzero_coefficients.gte(m), \
                                LARs_final_iteration(m, penultimate_outputs), penultimate_outputs))
        
        final_prediction = final_outputs.getArray('prediction')

        A = ee.List(final_outputs.get('A'))

        feature_path = A.slice(0, num_nonzero_coefficients).map(lambda index: feature_list.getString(index))        
        return feature_path

    
    select_features = ee_LARS(scaledImage, input_bandNames, responseBand, 5, 50000)
    #unclassified = ee.Image(cloud_folder+'/'+siteSelect+'_'+outputName+'_VI')
    unclassified = ee.Image(inputImage)
    # bands = ee.List([responseBand, 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI',
    #                  'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI', 'B2', 'B3', 'B4', 'B8',
    #                  'QA60', 'date', 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    bands = ee.List([responseBand, 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI',
                     'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI', 'B2', 'B3', 'B4', 'B8',
                     'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    unclassified = unclassified.rename(bands)

    # prediction bands (equivalent to select_features, with responseBand)
    bands = select_features
    input_bands = select_features.add(responseBand)
    training_data = ee.FeatureCollection(unclassified.sample(numPixels=1000, seed=1).select(input_bands))
    # implement regression tree with Random Forest algorithm
    # optional parameters for smileRandomForest(): variablesPerSplit, minLeafPopulation, bagFraction, maxNodes, seed
    rf_classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(features=training_data,
                                                                                           classProperty=responseBand,
                                                                                           inputProperties=input_bands)
    rf_classified = unclassified.select(bands).classify(rf_classifier, 'ALR_'+responseBand).clip(mapBounds)
    return temp_image.addBands(rf_classified)
    

---
## 1 – SL2P/SL2P10- processing NEON Simulated

## SL2P (input bands ('cosVZA', 'cosSZA', 'cosRAA',  'B3', 'B4', 'B5', 'B6', 'B7',  'B8A', 'B11', 'B12') 10m, output band 10m, use SL2P20) -Reference

In [ ]:
#ABBY
# users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m
# SJER
# users/GangHong2/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m
# JORN
# users/GangHong2/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m
#NOGP
# users/GangHong2/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m
#LENO
# users/GangHong2/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m
#MCDI
# users/GangHong2/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m
#UNDE
# users/GangHong2/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m
#STEI
# users/GangHong2/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m
#SERC
# users/GangHong2/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m
#HOPB
# users/GangHong2/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m

In [150]:
siteSelect='ABBY'
inputImg=ee.Image("users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m")
mapBounds =inputImg.geometry()
outputName = 'LAI'
assetfolder='users/GangHong2/NEON_result'

In [151]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [152]:
input_collection = ee.ImageCollection(inputImg).map(renameBands_NEON)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)   

image_output_names = ([name[16:]+"_"+outputName+"_"+"SL2P20" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [ ]:
print (image_output_names)

In [154]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                              
                                               image_names = image_output_names,
                                               scale=10,                                             
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (input bands ('cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8') 10m, output 10m,  using SL2P10) - no use in the process

In [ ]:
siteSelect='ABBY'
inputImg=ee.Image("users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m")
mapBounds =inputImg.geometry()
outputName = 'LAI'
assetfolder='users/GangHong2/NEON_result'

In [ ]:
# parse the networks
colName ='NEON_Sim_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [ ]:
input_collection_10m = ee.ImageCollection(inputImg).map(renameBands_NEON)

In [ ]:
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

In [ ]:
# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name[16:]+"_"+outputName+"_"+"SL2P10" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()])

ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [ ]:
print (image_output_names_10m )

In [ ]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,                                                 
                                                   image_names = image_output_names_10m,
                                                   scale=10,                                                 
                                                   asset_folder=assetfolder,                                             
                                                   data_type='float',
                                                   max_pixels=1e13)

## 2. Treatment N - S2NEON upscaled to 20m with the SL2P20 for prediction(null hypothesis )
### inputbands ('cosVZA', 'cosSZA', 'cosRAA', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 20m), ('B3', 'B4', 10m) and output bands 10m for comparison with the reference (10m)

In [ ]:
# The input list for each site  
# #ABBY
# "users/GangHong2/S2NEON/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m"
# "users/GangHong2/S2NEON/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m"
# #HOPB
# users/GangHong2/S2NEON/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m
# #JORN
# users/GangHong2/S2NEON/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m
# #LENO
# users/GangHong2/S2NEON/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m
# #MCDI
# users/GangHong2/S2NEON/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m
# #NOGP
# users/GangHong2/S2NEON/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m
# #SERC
# users/GangHong2/S2NEON/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m
# #SJER
# users/GangHong2/S2NEON/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m
# #STEI
# users/GangHong2/S2NEON/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m
# #UNDE
# users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m
# users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m

In [83]:
## Prepare imageries and parameters
# siteSelect='ABBY'
S2NEON_bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14']
S2NEON_new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
## change the input parameters based on the above block list for each site
S2NEON_10m=ee.Image('users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m').select(S2NEON_bands).rename(S2NEON_new_bands)
S2NEON_20m=ee.Image('users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m').select(S2NEON_bands).rename(S2NEON_new_bands)
## combine 20m bands and 10m bands to make an input
img_combined=S2NEON_20m.select( ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']).addBands(S2NEON_10m.select(['B2', 'B3', 'B4', 'B8']))
mapBounds=img_combined.geometry()
assetfolder='users/GangHong2/S2NEON_Result'
# assetfolder='users/GangHong2/S2NEON_ALR'

In [84]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [85]:
input_collection = ee.ImageCollection(img_combined)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
image_output_names = ([name[23:]+"_"+outputName+"_"+"SL2P20" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[23:], 23 here means the lengh of users/GangHong2/S2NEON/ 
## the following output for generate LAI with 20m as an input for ALR only
# image_output_names = ([name[23:]+"_"+outputName+"_"+"SL2P20_20m" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[23:], 23 here means the lengh of users/GangHong2/S2NEON/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [86]:
print (image_output_names)

['NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_LAI_SL2P20_20m']


In [87]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                              
                                               image_names = image_output_names,
                                               # scale=20,       ###  20 for LAI to be used in ALR
                                               scale=10,                                    
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## 3. Treatment A - S2NEON B,G,R,NIR with the SL2P10 for prediction
### input bands ('cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8', 10m) and output band 10m

In [ ]:
# The input list for each site  
# #ABBY
# "users/GangHong2/S2NEON/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m"
# #HOPB
# users/GangHong2/S2NEON/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m
# #JORN
# users/GangHong2/S2NEON/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m
# #LENO
# users/GangHong2/S2NEON/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m
# #MCDI
# users/GangHong2/S2NEON/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m
# #NOGP
# users/GangHong2/S2NEON/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m
# #SERC
# users/GangHong2/S2NEON/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m
# #SJER
# users/GangHong2/S2NEON/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m
# #STEI
# users/GangHong2/S2NEON/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m
# #UNDE
# users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m

In [155]:
## Prepare imageries and parameters
# siteSelect='UNDE'
S2NEON_bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14']
S2NEON_new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
# siteSelect='SERC'
S2NEON_10m=ee.Image('users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m').select(S2NEON_bands).rename(S2NEON_new_bands)
mapBounds=S2NEON_10m.geometry()
assetfolder='users/GangHong2/S2NEON_Result'

In [156]:
# parse the networks
colName ='NEON_Sim_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [157]:
input_collection_10m = ee.ImageCollection(S2NEON_10m)
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

In [158]:
# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name[23:]+"_"+outputName+"_"+"SL2P10_2" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()])

ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [159]:
print (image_output_names_10m)

['NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m_LAI_SL2P10_2']


In [160]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,                                           
                                                   image_names = image_output_names_10m,
                                                   scale=10,                                              
                                                   asset_folder=assetfolder,                                             
                                                   data_type='float',
                                                   max_pixels=1e13)

## 4. Treatment B - S2NEON downscaled to 10m using APTRK with SL2P20 for prediction
Input bands (downscaled 'B5', 'B6', 'B7',  'B8A', 'B11', 'B12' (10m) , 'B3' 'B4' (10m) and angles ('cosVZA', 'cosSZA', 'cosRAA'(10m)), the output is 10 for the comaprsion to the reference (10m)

In [ ]:
# The input list for each site  
# ABBY
# users/GangHong2/ATPRK/ABBY_Downscaling_ATPRK
# HOPB	
# users/GangHong2/ATPRK/HOPB_Downscaling_ATPRK
# JORN
# users/GangHong2/ATPRK/JORN_Downscaling_ATPRK
# LENO
# users/GangHong2/ATPRK/LENO_Downscaling_ATPRK
# MCDI
# users/GangHong2/ATPRK/MCDI_Downscaling_ATPRK
# NOGP
# users/GangHong2/ATPRK/NOGP_Downscaling_ATPRK
# SERC
# users/GangHong2/ATPRK/SERC_Downscaling_ATPRK
# SJER
# users/GangHong2/ATPRK/SJER_Downscaling_ATPRK
# STEI
# users/GangHong2/ATPRK/STEI_Downscaling_ATPRK
# UNDE
# users/GangHong2/ATPRK/UNDE_Downscaling_ATPRK


In [161]:
def  renameBands_ATPRK(image):
    bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11']
    new_bands = ['B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11','B12', 'cosVZA','cosSZA','cosRAA']
    return image.select(bands).rename(new_bands)

In [162]:
## Prepare imageries and parameters
## 10m Bands3,4 and angles have been added to the APTRK result and uploaded to GEE
# siteSelect='UNDE'
inputImg=ee.Image("users/GangHong2/ATPRK/UNDE_Downscaling_ATPRK")
assetfolder='users/GangHong2/ATPRK_RESULT'
mapBounds=inputImg.geometry()

In [163]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [164]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(inputImg).map(renameBands_ATPRK)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name +"_"+siteSelect +"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name[22:]+"_"+outputName+"_"+"SL2P20" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/NEON_FILT/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [165]:
print (image_output_names)

['UNDE_Downscaling_ATPRK_LAI_SL2P20']


In [ ]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,
                                               image_names = image_output_names,
                                               scale=10,                                               
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## 5. Treatment C - S2NEON with ALR for prediction
### First, use step 2 to generate a 20m output (LAI) instead of 10m as step 2. Second,  combine 10m bands ('B2','B3','B4','B8) as input.

In [ ]:
# #ABBY
# users/GangHong2/S2NEON_ALR/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m_LAI_SL2P20_20m
# "users/GangHong2/S2NEON/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m"
# #HOPB
# users/GangHong2/S2NEON_ALR/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m
# #JORN
# users/GangHong2/S2NEON_ALR/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m
# #LENO
# users/GangHong2/S2NEON_ALR/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m
# #MCDI
# users/GangHong2/S2NEON_ALR/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m
# #NOGP
# users/GangHong2/S2NEON_ALR/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m
# #SERC
# users/GangHong2/S2NEON_ALR/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m
# #SJER
# users/GangHong2/S2NEON_ALR/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m
# #STEI
# users/GangHong2/S2NEON_ALR/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m
# #UNDE
# users/GangHong2/S2NEON_ALR/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_LAI_SL2P20_20m
# users/GangHong2/S2NEON/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m

In [132]:
## Prepare imageries and parameters
S2NEON_bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14']
S2NEON_new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
# siteSelect='UNDE'
outputName = 'LAI'
# img_para should be a 20m LAI map made using 20m inputs to SL2P20m
# apply sl2P20m to the S2NEON_20m and DO NOT RESAMPLE UTPUT but leave at 20m
# img_para=ee.Image("users/GangHong2/S2NEON_Result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_LAI_SL2P20")
img_para=ee.Image("users/GangHong2/S2NEON_ALR/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_LAI_SL2P20_20m")
# this should be the S2NEON_20m but only bands so probably dont need all of this below just use img_para
img_10m=ee.Image('users/GangHong2/S2NEON/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m').select(S2NEON_bands).rename(S2NEON_new_bands)

# output_Name=assetname[21:]+'_ALR' ### index number neeed to be adjusted to get the meaningful file name '20210719T190919_20210719T191700_T10TER_ABBY_LAI_DenS2_20m_net_ALR'
img_combined=img_10m.select(['B2','B3','B4','B8']).addBands(img_para.select(['estimateLAI', 'partition', 'networkID', 'errorLAI', 'partition_1', 'networkID_1']))
mapBounds=img_combined.geometry()
# assetfolder='users/GangHong2/S2NEON_Result'
assetfolder='users/GangHong2/S2NEON_ALR'

In [133]:
responseBand = 'estimate'+outputName
ALR_result=func_ALR(img_combined, responseBand, outputName, mapBounds)
print (ALR_result.bandNames().getInfo())

['B2', 'B3', 'B4', 'B8', 'estimateLAI', 'partition', 'networkID', 'errorLAI', 'partition_1', 'networkID_1', 'ALR_estimateLAI']


In [134]:
name=img_para.get("system:id").getInfo()[27:-10]
print (name)

NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_LAI_


In [135]:
# export formatted image to GEE asset 
export = ee.ImageCollection(ALR_result)
export_task = ee_func.export_collection_to_gee(collection=export,
                                                 num_images=1,                                                                                                                                            
                                                 # image_names=[name+outputName+'_ALR'],
                                                 image_names=[name+'_ALR'],
                                                 # image_names=[output_Name],
                                                 asset_folder=assetfolder,                                                 
                                                 scale=10,
                                                 data_type='float',
                                                 max_pixels=1e13)

In [39]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(S2_selected).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
# image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_S2_ATPRK_20mnet" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_norm_ATPRK_"+outputName+"_SL2P20" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

## 6. Statistical analysis

### Treatments and the naming convention used for their input_list: 
reference - NEON upscaled to 10m with the SL2P20 for prediction<br>
Treatment N - S2NEON upscaled to 20m with the SL2P20 for prediction(null hypothesis )<br>
Treatment A - S2NEON B,G,R,NIR with the SL2P10 for prediction<br>
Treatment B ** - S2NEON downscaled to 10m using DSEN2 with SL2P20 for prediction<br>
Treatment C - S2NEON with ALR for prediction<br>
NEON = 1m NEON imagery converted to S2 spectral bands (previously)<br>
S2NEON = NEON imagery upscaled to S2 bands according to their point spread function (new with band names matching S2)<br>
SL2P20 = SL2P for all S2 input bands<br>
SL2P10 = SL2P for S2 B,G,R,NIR input bands only<br>

### statistics parameters used:
RMSE<br>
SSIM<br>
R2<br>

In [140]:
def statsComp(inputlist, para):
        ## stats for LAI from filter simulated NEON 20m
    df_all = pd.DataFrame(columns=['site','RMSE','SSIM','R2','bias'])
    ## a loop for each site
    for input in inputlist:     
        site=input.get('site')
        # print (site)
        ref_img=ee.Image(input.get('ref'))
        comp_img=ee.Image(input.get('comp'))
        ## make a new image to combine biophycal parameter from differnent data inputs
        input_img=ref_img.select('estimateLAI').rename('NEON_10m_LAI').addBands(comp_img.select(para).rename('Comp_10m_LAI'))
        ## generate sample points 
        samples_feat = ee.FeatureCollection(input_img.sample(numPixels=1000, seed=1))
        ## get the propery name of the sample points
        samples_col=samples_feat.first().propertyNames().getInfo()
        ## remove property named 'system:index'
        samples_col.remove('system:index')
        ## get a new data frame for featuure list
        sample_list = samples_feat.reduceColumns(ee.Reducer.toList(len(samples_col)), samples_col).values().get(0)
        df=pd.DataFrame(sample_list.getInfo(), columns=samples_col) 
        #print (df.isnull().sum().sum()) ## check the sum of null in dataframe
        ## get the column value, need to be divided by 1000
        df2=df[(df > 0).all(1)]
        ref_Vals = df2[samples_col[0]]/1000
        predict_Vals = df2[samples_col[1]]/1000
        ## mean_squared_error
        # RMSE = mean_squared_error(ref_Vals, predict_Vals)  
        RMSE = mean_squared_error(ref_Vals, predict_Vals,squared=False)  
        # RMSE = np.sqrt(np.mean(np.power(ref_Vals-predict_Vals,2)))
        # print (mean_squared_error(ref_Vals, predict_Vals,squared=False)) 
        bias=np.mean(ref_Vals-predict_Vals)
        ## SSIM
        SSIM = ssim(ref_Vals, predict_Vals, data_range=(predict_Vals.max() - predict_Vals.min())) 
      
        ## R square
        R2 = r2_score(ref_Vals, predict_Vals)        

        df_temp = pd.DataFrame([{'site':site, 'RMSE': RMSE, 'SSIM':SSIM,'R2': R2, 'bias': bias}])
        # print (df_temp)
        df_all= pd.concat([df_all, df_temp], axis=0, ignore_index=True)
    
    print (df_all)

### Comparison between reference and Treatment N (S2NEON upscaled to 20m with the SL2P20 for prediction(null hypothesis ))

In [5]:
## make a dictionry list for reference image and treatment N
input_list_N=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m_LAI_SL2P20'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m_LAI_SL2P20'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m_LAI_SL2P20'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m_LAI_SL2P20'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_LAI_SL2P20'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m_LAI_SL2P20'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_LAI_SL2P20'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m_LAI_SL2P20'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m_LAI_SL2P20'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m_LAI_SL2P20'}]


In [142]:
print ('*************************************************************************************************************************************')
print (' comparison between reference and Treatment N')
print ('*************************************************************************************************************************************')
statsComp(input_list_N,'estimateLAI')

*************************************************************************************************************************************
 comparison between reference and Treatment N
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.120224  0.917268  0.918189 -0.001105
1  SJER  0.283480  0.844114  0.808376 -0.010220
2  JORN  0.059430  0.780652  0.748764  0.004630
3  NOGP  0.326980  0.902886  0.873885  0.021962
4  LENO  0.251753  0.930518  0.910750  0.003431
5  MCDI  0.245464  0.897291  0.855440 -0.000344
6  UNDE  0.178787  0.914623  0.906544  0.013019
7  STEI  0.209784  0.910441  0.899561  0.009719
8  SERC  0.197827  0.888528  0.857065 -0.017779
9  HOPB  0.142343  0.812551  0.792622 -0.001864


## Comparison between reference and Treatment A (S2NEON B,G,R,NIR with the SL2P10 for prediction)

In [7]:
## make a dictionry list for reference image and treatment A
input_list_A=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m_LAI_SL2P10'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m_LAI_SL2P10'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m_LAI_SL2P10'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m_LAI_SL2P10'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m_LAI_SL2P10'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m_LAI_SL2P10'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m_LAI_SL2P10'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m_LAI_SL2P10'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m_LAI_SL2P10'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m_LAI_SL2P10'}]

In [143]:
print ('*************************************************************************************************************************************')
print (' Comparison between reference and Treatment A')
print ('*************************************************************************************************************************************')
statsComp(input_list_A,'estimateLAI')

*************************************************************************************************************************************
 Comparison between reference and Treatment A
*************************************************************************************************************************************
   site      RMSE      SSIM         R2      bias
0  ABBY  1.182824  0.444042  -6.952353 -1.004650
1  SJER  0.230517  0.920533   0.873694 -0.078923
2  JORN  0.268692  0.448766  -3.128416 -0.253263
3  NOGP  0.279672  0.923111   0.906505 -0.004261
4  LENO  0.369273  0.890781   0.704061 -0.194259
5  MCDI  0.398925  0.939400   0.617669 -0.336644
6  UNDE  0.671019  0.759933  -0.308813 -0.298068
7  STEI  0.595916  0.823469   0.197797 -0.416562
8  SERC  1.124456  0.547801  -3.555987 -0.924999
9  HOPB  1.530528  0.428148 -22.023135 -1.451814


## Comparison between reference and Treatment B (S2NEON downscaled to 10m using APTRK with SL2P20 for prediction)

In [9]:
## make a dictionry list for reference image and treatment B
input_list_B=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/ABBY_Downscaling_ATPRK_LAI_20m_net'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/SJER_Downscaling_ATPRK_LAI_20m_net'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/JORN_Downscaling_ATPRK_LAI_20m_net'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/NOGP_Downscaling_ATPRK_LAI_20m_net'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/LENO_Downscaling_ATPRK_LAI_20m_net'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/MCDI_Downscaling_ATPRK_LAI_20m_net'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/UNDE_Downscaling_ATPRK_LAI_20m_net'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/STEI_Downscaling_ATPRK_LAI_20m_net'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/SERC_Downscaling_ATPRK_LAI_20m_net'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/ATPRK_RESULT/HOPB_Downscaling_ATPRK_LAI_20m_net'}]


In [144]:
print ('*************************************************************************************************************************************')
print (' Comparison between reference and Treatment B')
print ('*************************************************************************************************************************************')
statsComp(input_list_B,'estimateLAI')

*************************************************************************************************************************************
 Comparison between reference and Treatment B
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.148936  0.897121  0.872753 -0.008882
1  SJER  0.237634  0.905867  0.866570 -0.005295
2  JORN  0.258542  0.666845 -2.197266 -0.032793
3  NOGP  0.262891  0.940096  0.917416 -0.013684
4  LENO  0.309483  0.918526  0.863765 -0.022212
5  MCDI  0.205303  0.935866  0.897947 -0.016480
6  UNDE  0.218514  0.912179  0.861201 -0.023085
7  STEI  0.158521  0.962483  0.942770 -0.018077
8  SERC  0.176392  0.915960  0.888419 -0.010080
9  HOPB  0.145315  0.862909  0.787742 -0.000771


## Comparison between reference and Treatment C (S2NEON with ALR for prediction)-old (using 20m output of LAI)

In [146]:
## make a dictionry list for reference image and treatment C
input_list_C0=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m_S2NEON_LAI_ALR'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_Result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m_S2NEON_LAI_ALR'}]


In [147]:
print ('*************************************************************************************************************************************')
print (' comparison between reference and Treatment C')
print ('*************************************************************************************************************************************')
statsComp(input_list_C0,'ALR_estimateLAI')

*************************************************************************************************************************************
 comparison between reference and Treatment C
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.129252  0.895262  0.905042 -0.003090
1  SJER  0.234330  0.897493  0.869482  0.001275
2  JORN  0.070500  0.789488  0.715783  0.007473
3  NOGP  0.247551  0.940679  0.927014  0.030671
4  LENO  0.216801  0.952733  0.934236  0.011127
5  MCDI  0.178678  0.946741  0.923299  0.016575
6  UNDE  0.174909  0.929147  0.911073  0.013353
7  STEI  0.204361  0.920237  0.905657  0.008561
8  SERC  0.203410  0.862001  0.851686  0.016678
9  HOPB  0.140984  0.765966  0.804647 -0.002028


## Comparison between reference and Treatment C (S2NEON with ALR for prediction)

In [148]:
## make a dictionry list for reference image and treatment C
input_list_C=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m_LAI_ALR'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m_LAI_ALR'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m_LAI_ALR'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m_LAI_ALR'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_LAI_ALR'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m_LAI_ALR'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_LAI_ALR'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m_LAI_ALR'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m_LAI_ALR'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2NEON_ALR/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m_LAI_ALR'}]

In [149]:
print ('*************************************************************************************************************************************')
print (' comparison between reference and Treatment C')
print ('*************************************************************************************************************************************')
statsComp(input_list_C,'ALR_estimateLAI')

*************************************************************************************************************************************
 comparison between reference and Treatment C
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.132137  0.881440  0.900757  0.002207
1  SJER  0.208516  0.919612  0.894835 -0.006041
2  JORN  0.066225  0.784736  0.749208  0.003297
3  NOGP  0.213919  0.952791  0.945498  0.022375
4  LENO  0.190413  0.963713  0.949271  0.006973
5  MCDI  0.168559  0.955832  0.931741  0.000378
6  UNDE  0.158591  0.940259  0.926892 -0.000481
7  STEI  0.196114  0.929285  0.913118 -0.002503
8  SERC  0.184987  0.883567  0.877335  0.000968
9  HOPB  0.135713  0.780118  0.818979 -0.014549
